In [1]:
import argparse
import requests
import numpy as np
import pandas as pd
import hail as hl
from hail.expr.functions import _sort_by
from hail.utils import hadoop_open, new_temp_file
import pyranges as pr
from ukbb_pan_ancestry import (
    get_distance_clumping_results_path,
    get_known_ukbb_loci_path,
    get_meta_analysis_results_path,
    get_munged_otg_v2d_path,
    get_munged_round2_path,
    get_pheno_manifest_path,
    get_ukb_pheno_efo_mapping_path,
    load_final_sumstats_mt,
    otg_release,
)

Loading BokehJS ...

In [ ]:
#--------------------------- 
# 09/20/2024
# expand 'region' to within 1Mb of lead variants
#---------------------------

In [2]:
def _to_pandas(ht):
            df = ht.to_pandas()
            # a temporary fix: cf. https://github.com/biocore-ntnu/pyranges/pull/264
            df = df.astype({k: "object" if v == "string[python]" else str(v).lower() for k, v in df.dtypes.items()})
            return df

In [ ]:
# otg lead variants
#ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
#ht_v2d = ht_v2d.annotate(
#    study_id=hl.delimit(ht_v2d.study_id),
#    Chromosome=ht_v2d.locus.contig,
#    Start=ht_v2d.locus.position,
#    End=ht_v2d.locus.position,
#)
#df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())
#gr_v2d = pr.PyRanges(df_v2d)
print(gr_v2d.head())

      locus    trait_efo           study_id                    trait_efo_term  \
0  1:768253  EFO_0006312       GCST90026371     mitochondrial DNA measurement   
1  1:839538  EFO_0006865  NEALE2_20002_1202      urgency urinary incontinence   
2  1:858051  EFO_0004617       GCST90025945            cystatin c measurement   
3  1:891059  EFO_0004533       GCST90019494  alkaline phosphatase measurement   
4  1:894573  EFO_0002618         GCST005434              pancreatic carcinoma   

         trait_efo_category Chromosome   Start     End  
0         Other measurement          1  768253  768253  
1               Other trait          1  839538  839538  
2         Other measurement          1  858051  858051  
3  Liver enzyme measurement          1  891059  891059  
4                    Cancer          1  894573  894573  
+------------+-------------+----------------------------------+-------+
| locus      | trait_efo   | study_id                         | +5    |
| (object)   | (object)    

In [ ]:
trait = "ALCO_AMOUNT"
# kcps2 lead variants
df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}.txt", delimiter="\t")
print(df)

# pyrange
gr = pr.PyRanges(df)
print(gr)

   idx    lead_locus                  region    trait_efo trait_efo_category  \
0    1    3:38052725     3:38052725-38052725  EFO_0007878  Other measurement   
1    2   4:100239319   4:100000136-100336448  EFO_0007878  Other measurement   
2    3    6:26207175     6:26180634-26326182  EFO_0007878  Other measurement   
3    4  12:108700164  12:108248685-108701635  EFO_0007878  Other measurement   
4    5  12:112241766  12:111414461-113117897  EFO_0007878  Other measurement   

   Chromosome      Start        End  
0           3   38052725   38052725  
1           4  100000136  100336448  
2           6   26180634   26326182  
3          12  108248685  108701635  
4          12  111414461  113117897  
+-----------+--------------+------------------------+-------------+-------+
|       idx | lead_locus   | region                 | trait_efo   | +4    |
|   (int64) | (object)     | (object)               | (object)    | ...   |
|-----------+--------------+------------------------+----------

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
gr['4', 100000136:100336445]

,idx,lead_locus,region,trait_efo,trait_efo_category,Chromosome,Start,End
1,2,4:100239319,4:100000136-100336448,EFO_0007878,Other measurement,4,100000136,100336448


In [12]:
df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
#df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])
print(df_annotated)

      idx    lead_locus                  region    trait_efo  \
0       1    3:38052725     3:38052725-38052725  EFO_0007878   
1       2   4:100239319   4:100000136-100336448  EFO_0007878   
2       2   4:100239319   4:100000136-100336448  EFO_0007878   
3       2   4:100239319   4:100000136-100336448  EFO_0007878   
4       2   4:100239319   4:100000136-100336448  EFO_0007878   
...   ...           ...                     ...          ...   
1259    5  12:112241766  12:111414461-113117897  EFO_0007878   
1260    5  12:112241766  12:111414461-113117897  EFO_0007878   
1261    5  12:112241766  12:111414461-113117897  EFO_0007878   
1262    5  12:112241766  12:111414461-113117897  EFO_0007878   
1263    5  12:112241766  12:111414461-113117897  EFO_0007878   

     trait_efo_category Chromosome      Start        End         locus  \
0     Other measurement          3   38052725   38052725            -1   
1     Other measurement          4  100000136  100336448   4:100006645   
2     Oth

In [7]:
ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
ht_v2d.show(3)

,,,,
locus,trait_efo,study_id,trait_efo_term,trait_efo_category
locus<GRCh37>,str,set<str>,str,str
1:768253,"""EFO_0006312""","{""GCST90026371""}","""mitochondrial DNA measurement""","""Other measurement"""
1:839538,"""EFO_0006865""","{""NEALE2_20002_1202""}","""urgency urinary incontinence""","""Other trait"""
1:858051,"""EFO_0004617""","{""GCST90025945""}","""cystatin c measurement""","""Other measurement"""


In [17]:
# otg lead variants
ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
ht_v2d = ht_v2d.annotate(
    study_id=hl.delimit(ht_v2d.study_id),
    Chromosome=ht_v2d.locus.contig,
    Start=ht_v2d.locus.position,
    End=ht_v2d.locus.position,
)
ht_v2d.show(3)

,,,,,,,
locus,trait_efo,study_id,trait_efo_term,trait_efo_category,Chromosome,Start,End
locus<GRCh37>,str,str,str,str,str,int32,int32
1:768253,"""EFO_0006312""","""GCST90026371""","""mitochondrial DNA measurement""","""Other measurement""","""1""",768253,768253
1:839538,"""EFO_0006865""","""NEALE2_20002_1202""","""urgency urinary incontinence""","""Other trait""","""1""",839538,839538
1:858051,"""EFO_0004617""","""GCST90025945""","""cystatin c measurement""","""Other measurement""","""1""",858051,858051


In [15]:
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

      locus    trait_efo           study_id                    trait_efo_term  \
0  1:768253  EFO_0006312       GCST90026371     mitochondrial DNA measurement   
1  1:839538  EFO_0006865  NEALE2_20002_1202      urgency urinary incontinence   
2  1:858051  EFO_0004617       GCST90025945            cystatin c measurement   
3  1:891059  EFO_0004533       GCST90019494  alkaline phosphatase measurement   
4  1:894573  EFO_0002618         GCST005434              pancreatic carcinoma   

         trait_efo_category Chromosome   Start     End  
0         Other measurement          1  768253  768253  
1               Other trait          1  839538  839538  
2         Other measurement          1  858051  858051  
3  Liver enzyme measurement          1  891059  891059  
4                    Cancer          1  894573  894573  


In [20]:
#========== EAS biobanks as ref ===========#
ht_v2d = hl.import_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/KoGES_otg_form.txt", delimiter='\t', impute=True)

# Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
ht_v2d = ht_v2d.annotate(locus_stripped = hl.str(ht_v2d.locus).replace("^Chr", ""))

# Parse the modified locus field to a proper Hail locus object
ht_v2d = ht_v2d.annotate(locus = hl.parse_locus(ht_v2d.locus_stripped, reference_genome='GRCh37'))

# Now you can access the contig and position attributes
ht_v2d = ht_v2d.annotate(
    study_id = ht_v2d.study_id,
    Chromosome = ht_v2d.locus.contig,
    Start = ht_v2d.locus.position,
    End = ht_v2d.locus.position
)
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

         locus    trait_efo      study_id trait_efo_term trait_efo_category  \
0  15:89804417  EFO_0004339  GCST90255478    body height        body height   
1  15:90141591  EFO_0004339  GCST90255478    body height        body height   
2  15:90619081  EFO_0004339  GCST90255478    body height        body height   
3  15:99192483  EFO_0004339  GCST90255478    body height        body height   
4  15:99240993  EFO_0004339  GCST90255478    body height        body height   

  locus_stripped Chromosome     Start       End  
0    15:89804417         15  89804417  89804417  
1    15:90141591         15  90141591  90141591  
2    15:90619081         15  90619081  90619081  
3    15:99192483         15  99192483  99192483  
4    15:99240993         15  99240993  99240993  


In [3]:
def annotate_known_loci_1Mb_EAS(trait):

    #========== EAS biobanks as ref ===========#
    ht_v2d = hl.import_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/KoGES_TWB_otg_form.txt", delimiter='\t', impute=True)

    # Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
    #ht_v2d = ht_v2d.annotate(locus_stripped=hl.str(ht_v2d.locus).replace("^Chr", ""))
    
    # Replace '23' with 'X' before parsing the locus
    ht_v2d = ht_v2d.annotate(
        locus=hl.if_else(ht_v2d.locus.startswith("23:"), 
                                  hl.str(ht_v2d.locus).replace("23:", "X:"), 
                                  ht_v2d.locus)
    )
    
    # Parse the modified locus field to a proper Hail locus object
    ht_v2d = ht_v2d.annotate(locus=hl.parse_locus(ht_v2d.locus, reference_genome='GRCh37'))
    
    # Now you can access the contig and position attributes
    ht_v2d = ht_v2d.annotate(
        study_id = ht_v2d.study_id,
        Chromosome = ht_v2d.locus.contig,
        Start = ht_v2d.locus.position,
        End = ht_v2d.locus.position
    )
    
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}_1Mb.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{trait}_1Mb_EAS.tsv")


In [4]:
def annotate_known_loci_500kb_EAS(trait):

    #========== EAS biobanks as ref ===========#
    ht_v2d = hl.import_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/KoGES_TWB_otg_form.txt", delimiter='\t', impute=True)

    # Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
    #ht_v2d = ht_v2d.annotate(locus_stripped=hl.str(ht_v2d.locus).replace("^Chr", ""))
    
    # Replace '23' with 'X' before parsing the locus
    ht_v2d = ht_v2d.annotate(
        locus=hl.if_else(ht_v2d.locus.startswith("23:"), 
                                  hl.str(ht_v2d.locus).replace("23:", "X:"), 
                                  ht_v2d.locus)
    )
    
    # Parse the modified locus field to a proper Hail locus object
    ht_v2d = ht_v2d.annotate(locus=hl.parse_locus(ht_v2d.locus, reference_genome='GRCh37'))
    
    # Now you can access the contig and position attributes
    ht_v2d = ht_v2d.annotate(
        study_id = ht_v2d.study_id,
        Chromosome = ht_v2d.locus.contig,
        Start = ht_v2d.locus.position,
        End = ht_v2d.locus.position
    )
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}_500kb.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{trait}_500kb_EAS.tsv")


In [ ]:
## 1Mb

In [18]:
trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "INSULIN","GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCHC", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
              "HEIGHT", "BMI", "WAIST", "SMOKA_MOD", "ALCO_AMOUNT","COFFA"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_1Mb_EAS(trait)    

LDL


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15539259""","""EFO_0004518""","[""GCST90255485""]","""creatinine measurement""","""creatinine measurement""",1:15539259
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15782677""","""EFO_0004518""","[""GCST90255485""]","""creatinine measurement""","""creatinine measurement""",1:15782677
1,1:16505320,[1:15505320-1:17505320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:15816768""","""EFO_0004518""","[""GCST90255485""]","""creatinine measurement""","""creatinine measurement""",1:15816768


trait_efo
62
46
0
0
trait_efo_category
0
108
0
0


TG


...

trait_efo
3
2
0
0
trait_efo_category
0
5
0
0


COFFA


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88022709""","""EFO_0004612""","[""GCST90255491""]","""high density lipoprotein cholesterol measurement""","""high density lipoprotein cholesterol measurement""",4:88022709
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88039000""","""EFO_0004570""","[""GCST90255487""]","""bilirubin measurement""","""bilirubin measurement""",4:88039000
1,4:89020503,[4:88020503-4:90020503),"""EFO_0006781""","""Other measurement""","""4:88040909""","""EFO_0004348""","[""GCST90255496""]","""hematocrit""","""hematocrit""",4:88040909


trait_efo
0
3
0
0
trait_efo_category
0
3
0
0


In [ ]:
## 500kb

In [20]:
trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "INSULIN","GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCHC", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
              "HEIGHT", "BMI", "WAIST", "SMOKA_MOD", "ALCO_AMOUNT","COFFA"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_500kb_EAS(trait)    

LDL


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16184747""","""EFO_0003923""","[""GCST90278620""]","""bone density""","""bone density""",1:16184747
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16184747""","""EFO_0003923""","[""GCST90278621""]","""bone density""","""bone density""",1:16184747
1,1:16505320,[1:16005320-1:17005320),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16364255""","""EFO_0004518""","[""GCST90255485""]","""creatinine measurement""","""creatinine measurement""",1:16364255


trait_efo
58
50
0
0
trait_efo_category
0
108
0
0


TG


...

trait_efo
3
2
0
0
trait_efo_category
0
5
0
0


COFFA


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88692427""","""EFO_0004735""","[""GCST90255488""]","""serum alanine aminotransferase measurement""","""serum alanine aminotransferase measurement""",4:88692427
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88768860""","""EFO_0003923""","[""GCST90278620""]","""bone density""","""bone density""",4:88768860
1,4:89020503,[4:88520503-4:89520503),"""EFO_0006781""","""Other measurement""","""4:88768860""","""EFO_0003923""","[""GCST90278621""]","""bone density""","""bone density""",4:88768860


trait_efo
0
3
0
0
trait_efo_category
0
3
0
0


In [ ]:
# 10/05/2024
# After correcting EFO_ID for FBS

In [5]:
trait_list = ["FBS"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_500kb_EAS(trait)  

Initializing Hail with default parameters...


FBS
24/10/06 00:25:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://holy7c04105.rc.fas.harvard.edu:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/hail-20241006-0025-0.2.130-bea04d9c79b5.log
2024-10-06 00:25:25.952 Hail: INFO: Reading table to impute column types
2024-10-06 00:25:27.198 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is de

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43455283""","""EFO_0004465""","[""GCST90278628""]","""fasting blood glucose measurement""","""fasting blood glucose measurement""",1:43455283
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43805737""","""EFO_0004308""","[""GCST90278647""]","""leukocyte count""","""leukocyte count""",1:43805737
1,1:43453188,[1:42953188-1:43953188),"""EFO_0004465""","""Other measurement""","""1:43884714""","""EFO_0004309""","[""GCST90255498""]","""platelet count""","""platelet count""",1:43884714


trait_efo
44
43
0
0
trait_efo_category
0
87
0
0


2024-10-06 00:25:53.038 Hail: INFO: merging 1001 files totalling 232.1K... 1000]
2024-10-06 00:25:53.179 Hail: INFO: while writing:
    /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_FBS_500kb_EAS.tsv
  merge time: 138.290ms
2024-10-06 00:26:36.188 Hail: INFO: Reading table to impute column types
2024-10-06 00:26:36.650 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
2024-10-06 00:26:38.823 Hail: INFO: Reading table to impute column types
2024-10-06 00:26:46.464 Hail: INFO: Finished type imputation
  Loading field 'idx' as type int32 (imputed)
  Loading field 'lead_locus' as type str (imputed)
  Loading field 'region' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'trait_efo_categ

In [7]:
trait_list = ["FBS"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_1Mb_EAS(trait)  

FBS


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:43455283""","""EFO_0004465""","[""GCST90278628""]","""fasting blood glucose measurement""","""fasting blood glucose measurement""",1:43455283
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:43805737""","""EFO_0004308""","[""GCST90278647""]","""leukocyte count""","""leukocyte count""",1:43805737
1,1:43453188,[1:42453188-1:44453188),"""EFO_0004465""","""Other measurement""","""1:43884714""","""EFO_0004309""","[""GCST90255498""]","""platelet count""","""platelet count""",1:43884714


trait_efo
44
43
0
0
trait_efo_category
0
87
0
0


In [ ]:
# tmp - distance clumping

In [ ]:
def agg_distance_clumping(
    locus_expr, alleles_expr, pvalue_expr, radius=500000, nlog_p=False, merge_overlapping_loci=True
):
    def _distance_clumping(f, lead_st_arr, remaining_st_arr):
        new_lead_st = remaining_st_arr[0]
        new_region = new_lead_st.region
        remaining_st_arr = remaining_st_arr.filter(lambda x: ~new_region.contains(x.lead_locus))

        # check whether a new region overlaps with previous ones
        overlapping_st = (
            lead_st_arr.find(lambda x: new_region.overlaps(x.region))
            if merge_overlapping_loci
            else hl.missing(new_lead_st.dtype)
        )
        lead_st_arr = hl.if_else(
            hl.is_defined(overlapping_st),
            lead_st_arr.map(
                lambda x: hl.if_else(
                    x == overlapping_st,
                    x.annotate(
                        region=hl.rbind(
                            overlapping_st.region,
                            lambda old_region: hl.interval(
                                hl.if_else(
                                    old_region.start < new_region.start,
                                    old_region.start,
                                    new_region.start,
                                ),
                                hl.if_else(old_region.end > new_region.end, old_region.end, new_region.end),
                            ),
                        )
                    ),
                    x,
                )
            ),
            lead_st_arr.append(new_lead_st),
        )
        converged = hl.len(remaining_st_arr) == 0

        return hl.if_else(converged, lead_st_arr, f(lead_st_arr, remaining_st_arr))

    # Modified from hl.expr.functions.sorted -- key is now another collection, not callable
    def _sorted(collection, key, reverse=False):
        def comp(left, right):
            return (
                hl.case()
                .when(hl.is_missing(left), False)
                .when(hl.is_missing(right), True)
                .when(reverse, hl._compare(right, left) < 0)
                .default(hl._compare(left, right) < 0)
            )

        return _sort_by(hl.zip(key, collection), lambda l, r: comp(l[0], r[0])).map(lambda elt: elt[1])

    # lead locus / region pair is necessary to check overlapping regions during the loop
    x = hl.agg.collect(
        hl.struct(
            st=hl.struct(
                lead_locus=locus_expr,
                lead_alleles=alleles_expr,
                region=locus_expr.window(radius, radius),
                lead_pvalue=pvalue_expr,
            )
        )
    )
    sorted_st = _sorted(x.st, x.st.lead_pvalue, reverse=nlog_p)
    t_struct = sorted_st[0].dtype

    run_distance_clumping = hl.experimental.define_function(
        lambda sorted_st: hl.experimental.loop(
            _distance_clumping, hl.tarray(t_struct), hl.empty_array(t_struct), sorted_st
        ),
        hl.tarray(t_struct),
    )
    return run_distance_clumping(sorted_st)


In [ ]:
ht = ht.group_by(*list(mt.col_key)).aggregate(
            distance_clumps=agg_distance_clumping(
                ht.locus,
                ht.alleles,
                ht.Pvalue,
                radius=args.radius,
                nlog_p=False,
                merge_overlapping_loci=(not args.not_merge_overlapping_loci),
            )
        )